In [1]:
import torch
import pickle
import numpy as np
import pandas as pd
import os

from os.path import dirname



root_path = dirname(os.getcwd()) + "/HGNN_NA"

pd.set_option("display.max_columns", None)
data_dir = root_path + "/data/datasets/original/"
data_dir_processed = root_path + "/data/datasets/processed/"
data_dir_graphs = root_path + "/data/datasets/graphs/"

print(root_path, data_dir, data_dir_processed, data_dir_graphs, sep="\n")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"

/home/sebdis/ProcessMining/HGNN/HGNN_NA
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/original/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/processed/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/graphs/


In [2]:
dataset = "BPI_Challenge_2013_open_problems"

In [3]:
categorical_columns = ['org:group', 'resource country', 'org:resource', 'oranization country',
       'org:role', 'Activity', 'impact', 'product', "org:resource:role"]
real_value_columns = ["time:timestamp"]

In [4]:
tab_all = pd.read_csv(data_dir_processed+dataset+"_processed_all.csv")
tab_all.head()

,org:group,resource country,org:resource,oranization country,org:role,Activity,impact,product,time:timestamp,CaseID,org:resource:role
0,Org line A2,Sweden,Tomas,cn,A2_2,Accepted_In Progress,Medium,PROD753,0.000000,1-147898401,Role 1
1,Org line A2,Sweden,Tomas,cn,A2_2,Accepted_In Progress,Medium,PROD753,9.315421,1-147898401,Role 1
2,Org line A2,Sweden,Tomas,cn,A2_2,Accepted_Wait,Medium,PROD753,18.388883,1-147898401,Role 1
3,Org line A2,Sweden,Tomas,cn,A2_2,Accepted_In Progress,Medium,PROD753,18.840289,1-147898401,Role 1
4,Org line A2,Sweden,Tomas,cn,A2_2,Accepted_In Progress,Medium,PROD753,16.256809,1-165554831,Role 1


In [5]:
from data import utils

one_hot_resource = utils.get_one_hot_encoder(tab_all, "org:resource")
one_hot_resource_role = utils.get_one_hot_encoder(tab_all, "org:resource:role")
resources = tab_all["org:resource"]
resources_role = tab_all["org:resource:role"]
map_resource_to_role = {}
for i in range(len(resources)):
    map_resource_to_role[resources[i]] = resources_role[i]

map_resource_to_role

{'Tomas': 'Role 1',
 'Niklas': 'Role 1',
 'Ewa': 'Role 1',
 'Pawel': 'Role 1',
 'Panigrahy': 'Role 1',
 'Jerker': 'Role 1',
 'Srinivasan': 'Role 1',
 'Aneesh V': 'Role 1',
 'Rijin': 'Role 1',
 'Celine': 'Role 1',
 'Anna': 'Role 1',
 'Sumesh': 'Role 1',
 'Prasad': 'Role 1',
 'Peter': 'Role 1',
 'Craig': 'Role 1',
 'Stefan': 'Role 1',
 'David': 'Role 1',
 'Jonas': 'Role 1',
 'Joakim': 'Role 1',
 'Per': 'Role 1',
 'Stephen': 'Role 1',
 'Christer': 'Role 1',
 'Ing-Marie': 'Role 1',
 'Nicolas': 'Role 1',
 'Kenneth': 'Role 1',
 'Fredrik': 'Role 1',
 'Viktoria': 'Role 1',
 'Roland': 'Role 1',
 'Marco': 'Role 1',
 'Katarina': 'Role 1',
 'Ian': 'Role 1',
 'Martin': 'Role 1',
 'Olivier': 'Role 1',
 'Britt': 'Role 1',
 'Michal': 'Role 1',
 'Timothy': 'Role 1',
 'Andrew': 'Role 1',
 'Mikael': 'Role 1',
 'Erik': 'Role 1',
 'Arun': 'Role 1',
 'Els': 'Role 1',
 'Jo': 'Role 1',
 'Lars': 'Role 1',
 'Daniel': 'Role 1',
 'Rickard': 'Role 1',
 'Thiago': 'Role 1',
 'Cyril': 'Role 1',
 'Mattias': 'Role 1',


In [6]:
map_resource_index = {}
for x in tab_all["org:resource"].unique():
    map_resource_index[x] = torch.argmax(torch.tensor(utils.get_one_hot_encodings(one_hot_resource, np.array([x])))).item()
map_resource_index

{'Tomas': 223,
 'Niklas': 159,
 'Ewa': 57,
 'Pawel': 170,
 'Panigrahy': 163,
 'Jerker': 93,
 'Srinivasan': 209,
 'Aneesh V': 12,
 'Rijin': 191,
 'Celine': 36,
 'Anna': 15,
 'Sumesh': 217,
 'Prasad': 175,
 'Peter': 172,
 'Craig': 44,
 'Stefan': 210,
 'David': 47,
 'Jonas': 105,
 'Joakim': 99,
 'Per': 171,
 'Stephen': 213,
 'Christer': 38,
 'Ing-Marie': 83,
 'Nicolas': 158,
 'Kenneth': 114,
 'Fredrik': 63,
 'Viktoria': 234,
 'Roland': 195,
 'Marco': 138,
 'Katarina': 110,
 'Ian': 81,
 'Martin': 145,
 'Olivier': 160,
 'Britt': 31,
 'Michal': 150,
 'Timothy': 222,
 'Andrew': 11,
 'Mikael': 152,
 'Erik': 55,
 'Arun': 18,
 'Els': 51,
 'Jo': 97,
 'Lars': 123,
 'Daniel': 46,
 'Rickard': 190,
 'Thiago': 220,
 'Cyril': 45,
 'Mattias': 148,
 'Bharath': 27,
 'Praveen': 177,
 'Steve': 214,
 'Murali': 156,
 'Rohan': 193,
 'Wim': 237,
 'Inger': 85,
 'Frederic': 62,
 'Miroslaw': 154,
 'Lena': 128,
 'Radoslaw': 179,
 'Robert': 192,
 'Anup': 17,
 'Lars-Ove': 125,
 'Jörgen': 109,
 'Hineesh': 78,
 'Carlos

In [7]:
map_resource_role_index = {}
for x in tab_all["org:resource:role"].unique():
    map_resource_role_index[x] = torch.argmax(torch.tensor(utils.get_one_hot_encodings(one_hot_resource_role, np.array([x])))).item()
map_resource_role_index

{'Role 1': 0, 'Role 2': 1}

In [8]:
map_resource_to_role = { map_resource_index[k] : map_resource_role_index[v] for k,v in map_resource_to_role.items()}
map_resource_to_role

{223: 0,
 159: 0,
 57: 0,
 170: 0,
 163: 0,
 93: 0,
 209: 0,
 12: 0,
 191: 0,
 36: 0,
 15: 0,
 217: 0,
 175: 0,
 172: 0,
 44: 0,
 210: 0,
 47: 0,
 105: 0,
 99: 0,
 171: 0,
 213: 0,
 38: 0,
 83: 0,
 158: 0,
 114: 0,
 63: 0,
 234: 0,
 195: 0,
 138: 0,
 110: 0,
 81: 0,
 145: 0,
 160: 0,
 31: 0,
 150: 0,
 222: 0,
 11: 0,
 152: 0,
 55: 0,
 18: 0,
 51: 0,
 97: 0,
 123: 0,
 46: 0,
 190: 0,
 220: 0,
 45: 0,
 148: 0,
 27: 0,
 177: 0,
 214: 0,
 156: 0,
 193: 0,
 237: 0,
 85: 0,
 62: 0,
 154: 0,
 128: 0,
 179: 0,
 192: 0,
 17: 0,
 125: 0,
 109: 0,
 78: 0,
 35: 0,
 1: 0,
 14: 0,
 147: 0,
 161: 0,
 189: 0,
 204: 0,
 233: 0,
 121: 0,
 2: 0,
 185: 0,
 227: 0,
 33: 0,
 129: 0,
 10: 0,
 203: 0,
 166: 0,
 89: 0,
 32: 0,
 4: 0,
 0: 0,
 231: 0,
 61: 0,
 167: 0,
 139: 0,
 118: 0,
 29: 0,
 141: 0,
 230: 0,
 151: 0,
 184: 0,
 127: 0,
 183: 0,
 228: 0,
 116: 0,
 96: 0,
 80: 0,
 58: 0,
 224: 0,
 169: 0,
 122: 0,
 136: 0,
 206: 0,
 6: 0,
 90: 0,
 182: 0,
 52: 0,
 202: 0,
 16: 0,
 120: 0,
 173: 0,
 126: 0,
 200:

In [9]:
list_unique = {k : list(tab_all[k].unique()) for k in categorical_columns}
list_unique

{'org:group': ['Org line A2',
  'Org line G3',
  'Org line G4',
  'Org line B',
  'Org line C',
  'Org line F',
  'Org line D',
  'Org line V2',
  'Org line V11',
  'Org line G1',
  'Org line V5'],
 'resource country': ['Sweden',
  'POLAND',
  'INDIA',
  'France',
  '0',
  'USA',
  'United Kingdom',
  'Belgium',
  'Brazil',
  'China',
  'Japan',
  'Denmark',
  'Australia',
  'Canada'],
 'org:resource': ['Tomas',
  'Niklas',
  'Ewa',
  'Pawel',
  'Panigrahy',
  'Jerker',
  'Srinivasan',
  'Aneesh V',
  'Rijin',
  'Celine',
  'Anna',
  'Sumesh',
  'Prasad',
  'Peter',
  'Craig',
  'Stefan',
  'David',
  'Jonas',
  'Joakim',
  'Per',
  'Stephen',
  'Christer',
  'Ing-Marie',
  'Nicolas',
  'Kenneth',
  'Fredrik',
  'Viktoria',
  'Roland',
  'Marco',
  'Katarina',
  'Ian',
  'Martin',
  'Olivier',
  'Britt',
  'Michal',
  'Timothy',
  'Andrew',
  'Mikael',
  'Erik',
  'Arun',
  'Els',
  'Jo',
  'Lars',
  'Daniel',
  'Rickard',
  'Thiago',
  'Cyril',
  'Mattias',
  'Bharath',
  'Praveen',
 

In [10]:
import random

torch.manual_seed(0)
torch.cuda.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [11]:
with open(data_dir_graphs + dataset + "_TRAIN_event_prediction_FINAL.pkl", "rb") as f:
    X_train = pickle.load(f)
with open(data_dir_graphs + dataset + "_VALID_event_prediction_FINAL.pkl", "rb") as f:
    X_valid = pickle.load(f)
with open(data_dir_graphs + dataset + "_TEST_event_prediction_FINAL.pkl", "rb") as f:
    X_test = pickle.load(f)

In [12]:

from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import ToUndirected, NormalizeFeatures

transform = ToUndirected()

with torch.no_grad():
        for i in range(len(X_train)):
                X_train[i] = transform(X_train[i])
        for i in range(len(X_valid)):
                X_valid[i] = transform(X_valid[i])
        for i in range(len(X_test)):
                X_test[i] = transform(X_test[i])
    






In [13]:
edge_types = set()
node_types = set()
for i in range(len(X_train)):
    n, edge_type = X_train[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)
for i in range(len(X_valid)):
    n, edge_type = X_valid[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)
for i in range(len(X_test)):
    n, edge_type = X_test[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)



In [14]:
node_types = list(node_types)
edge_types = list(edge_types)

In [ ]:
node_types

In [ ]:
edge_types

In [15]:
def get_weights(load, key):
    weights = []
    
    cl_train = [0 for _ in tab_all[key].unique()]
    
    print(cl_train)
    
    for i,x in enumerate(load):

        
        classes = x.y[key]

        # print(classes)
        
        for c in list(classes):
            try:
                cl_train[c] +=1
            except KeyError:
                cl_train[c] = 1
    s = sum(cl_train)
    
    print(cl_train)
    
    weights = [s/x if x > 0 else 0 for x in cl_train]

    # weights = [0.7,0.7,1,0.7,0.7,0.7,0.7,0.7,0.7,0.7]
    weights = torch.tensor(weights, device=device)
    print(weights)
    return weights
    

In [16]:
edge_features_dims = {}

for i in range(len(X_train)):
    for k in X_train[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_train[i].edge_attr_dict[k].shape[1]
for i in range(len(X_valid)):
    for k in X_train[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_train[i].edge_attr_dict[k].shape[1]
for i in range(len(X_test)):
    for k in X_train[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_train[i].edge_attr_dict[k].shape[1]

edge_features_dims

{('Activity', 'followed_by', 'Activity'): 2,
 ('time:timestamp', 'related_to', 'time:timestamp'): 2,
 ('org:resource', 'related_to', 'org:resource'): 2,
 ('resource country', 'related_to', 'resource country'): 2,
 ('org:group', 'related_to', 'org:group'): 2,
 ('org:role', 'related_to', 'org:role'): 2}

In [17]:
act_weights = get_weights(DataLoader(X_train, batch_size=1024, shuffle=False), "Activity")
# res_roles_weights = get_weights(DataLoader(X_train, batch_size=1024, shuffle=False), "org:resource:role")


[0, 0, 0, 0, 0]
[52, 202, 57, 185, 95]
tensor([11.3654,  2.9257, 10.3684,  3.1946,  6.2211], device='cuda:0')


## Hyperopt

In [18]:
from ax.service.managed_loop import optimize

In [19]:
from torch_geometric.nn import (
    HeteroConv,
    global_mean_pool,
    GATv2Conv
)
from torch.nn import (
    ModuleList,
    Module,
    Linear
  )
from typing_extensions import Self

In [20]:
class HGNN(Module):

    def __init__(self, output_cat, output_real,nodes_relations, relations_with_features, parameters) -> Self:  # type: ignore
        super().__init__()

        # List of convolutional layers
        
        hid = parameters["hid"]
        layers = parameters["layers"]
        aggregation = parameters["aggregation"]
        n_heads = parameters["heads"]
        
        self.output_cat = output_cat
        self.output_real = output_real
        
        self.convs = ModuleList()
        for _ in range(layers):
            conv = HeteroConv(
                {
                    relation: (
                        GATv2Conv((-1,-1), add_self_loops=False, out_channels=hid, heads=n_heads, concat=False)
                        if relation not in relations_with_features
                        else GATv2Conv((-1,-1), add_self_loops=False, out_channels=hid,heads=n_heads, edge_dim=relations_with_features[relation], concat=False)
                    )
                    for relation in nodes_relations
                },
                aggr=aggregation,
            )

            self.convs.append(conv)

        self.FC = {}
        
        for k in output_cat:
            self.FC[k] = Linear(hid, output_cat[k], device=device)
        for k in output_real:
            self.FC[k] = Linear(hid, 1, device=device)
        
    
        
        

    def forward(self, batch):

        for i in range(len(self.convs)):
            batch.x_dict = self.convs[i]( 
                batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict
            )

            batch.x_dict = {key: x.relu() for key, x in batch.x_dict.items()}


        output = {}
        
        for k in self.output_cat:
            output[k] = global_mean_pool(batch.x_dict[k], batch[k].batch)
            output[k] = self.FC[k](output[k])
        for k in self.output_real:
            output[k] = global_mean_pool(batch.x_dict[k], batch[k].batch)
            output[k] = self.FC[k](output[k]).reshape(1,-1)[0]
            

        return output

In [21]:
from torcheval.metrics.functional import multiclass_accuracy, multiclass_f1_score

In [22]:
import torch.nn as nn

In [23]:
import time

In [24]:
from copy import deepcopy


def train_hgnn(config, output_cat, output_real, epochs=20):
    print(config)

    net = HGNN(
        parameters=config,
        output_cat=output_cat,
        output_real=output_real,
        nodes_relations=edge_types,
        relations_with_features=edge_features_dims,
    )
    net = net.to(device)

    losses = {}

    for k in output_cat:
        losses[k] = (
            nn.CrossEntropyLoss()
            if k != "Activity"
            else nn.CrossEntropyLoss(act_weights)
        )
    for k in output_real:
        losses[k] = nn.L1Loss()

    train_loader = DataLoader(X_train, batch_size=config["batch_size"], shuffle=True)
    valid_loader = DataLoader(X_valid, batch_size=config["batch_size"], shuffle=True)

    optimizer = torch.optim.Adam(net.parameters(), lr=config["lr"])

    best_model = None
    best_loss = 0
    patience = 10
    pat_count = 0

    torch.cuda.empty_cache()

    for epoch in range(0, epochs):
        start_time = time.time()

        print(f"Epoch: {epoch}\n")

        net.train()
        for _, x in enumerate(train_loader):
            x = x.to(device)

            labels = x.y

            optimizer.zero_grad()
            outputs = net(x)

            losses_step = {k: losses[k](outputs[k], labels[k]) for k in losses}

            total_loss = 0
            for k in losses_step:
                total_loss += losses_step[k]

            total_loss.backward()
            optimizer.step()

        predictions_categorical = {k: [] for k in output_cat}
        target_categorical = {k: [] for k in output_cat}

        avg_MAE = {k: [] for k in output_real}

        running_total_loss = []

        net.eval()
        with torch.no_grad():
            for i, x in enumerate(valid_loader):
                x = x.to(device)

                labels = x.y

                outputs = net(x)

                losses_step = {k: losses[k](outputs[k], labels[k]) for k in losses}

                running_total_loss.append(sum(list(losses_step.values())))

                for k in output_cat:
                    predictions_categorical[k].append(
                        torch.argmax(torch.softmax(outputs[k], dim=1), 1)
                    )
                    target_categorical[k].append(labels[k])

                for k in output_real:
                    avg_MAE[k].append(losses_step[k])

        for k in predictions_categorical:
            predictions_categorical[k] = torch.cat(predictions_categorical[k])
            target_categorical[k] = torch.cat(target_categorical[k])

        macro_f1_activity = multiclass_f1_score(
            predictions_categorical["Activity"],
            target_categorical["Activity"],
            num_classes=output_cat["Activity"],
            average="macro",
        )

        accuracy = {
            k: multiclass_accuracy(
                predictions_categorical[k],
                target_categorical[k],
                num_classes=output_cat[k],
            )
            for k in output_cat
        }

        avg_MAE = {k: sum(avg_MAE[k]) / len(avg_MAE[k]) for k in avg_MAE}

        val_loss = sum(running_total_loss) / len(running_total_loss)

        print(f"\nVALIDATION")
        for k in accuracy:
            (
                print("{}: acc {:.4f}".format(k, accuracy[k]))
                if k != "Activity"
                else print(
                    "{}: acc {:.4f} macroF1 {:.4f}".format(
                        k, accuracy[k], macro_f1_activity.item()
                    )
                )
            )
        for k in avg_MAE:
            print("{}: MAE {:.4f}".format(k, avg_MAE[k]))
        print("TOTAL_LOSS: {:.4f}".format(val_loss))
        print("epoch time {}s\n".format(time.time() - start_time))

        if epoch == 0:
            best_model = deepcopy(net)
            best_loss = val_loss
        else:
            if val_loss < best_loss:
                best_loss = val_loss
                best_model = deepcopy(net)
                pat_count = 0
                print("new best model found")
            if pat_count == patience:
                print(
                    "Validation performance didn't improve for {} epochs. Training stops.".format(
                        pat_count
                    )
                )
                return best_model
        pat_count += 1

    return best_model

In [25]:
def test_hgnn(net, output_cat, output_real):
    test_loader = DataLoader(X_test, batch_size=128, shuffle=False)
    
    losses = {}
    
    for k in output_cat:
        losses[k] = (
            nn.CrossEntropyLoss()
            if k != "Activity"
            else nn.CrossEntropyLoss(act_weights)
        )
    for k in output_real:
        losses[k] = nn.L1Loss()
    
    
    
    
    
    predictions_categorical = {k: [] for k in output_cat}
    target_categorical = {k: [] for k in output_cat}

    avg_MAE = {k : [] for k in output_real}
    
    total_loss = []
        
    net.eval()
    with torch.no_grad():
        for i, x in enumerate(test_loader):
            x = x.to(device)
            
            labels = x.y
            
            outputs = net(x)
            
     
            losses_step = {k: losses[k](outputs[k], labels[k]).item() for k in losses}
            total_loss.append(sum(list(losses_step.values())))
            
            for k in output_cat:
                    predictions_categorical[k].append(
                        torch.argmax(torch.softmax(outputs[k], dim=1), 1)
                    )
                    target_categorical[k].append(labels[k])
            
            
            for k in output_real:
                    avg_MAE[k].append(losses_step[k])
                    
    for k in predictions_categorical:
            predictions_categorical[k] = torch.cat(predictions_categorical[k])
            target_categorical[k] = torch.cat(target_categorical[k])
               
            
    macro_f1_activity = multiclass_f1_score(
            predictions_categorical["Activity"],
            target_categorical["Activity"],
            num_classes=output_cat["Activity"],
            average="macro",
        )
            
    accuracy = {
            k: multiclass_accuracy(
                predictions_categorical[k],
                target_categorical[k],
                num_classes=output_cat[k],
            )
            for k in output_cat
        }
    
    resource_to_role_acc = multiclass_accuracy(
        torch.tensor([map_resource_to_role[x.item()] for x in predictions_categorical["org:resource"]], device=device),
        target_categorical["org:resource:role"],
        num_classes=output_cat["org:resource:role"]
    )
    
    avg_MAE = {k : sum(avg_MAE[k]) / len(avg_MAE[k]) for k in avg_MAE}
    
    
    Average_total_loss = sum(total_loss) / len(total_loss)
    
    res = {f"{k}_acc" : accuracy[k].item() for k in accuracy} | {"Resource_to_role_acc" : resource_to_role_acc.item()} |{"MacroF1Act" : macro_f1_activity.item()} | {f"{k}_mae" : avg_MAE[k] for k in avg_MAE} | {"AVG_total_loss" : Average_total_loss} 
    
    #print(res)
    
    return res

In [26]:
outputcat = {k : len(list_unique[k]) for k in list_unique}
outputreal = real_value_columns
print(outputcat)
print(outputreal)

{'org:group': 11, 'resource country': 14, 'org:resource': 240, 'oranization country': 14, 'org:role': 26, 'Activity': 5, 'impact': 4, 'product': 139, 'org:resource:role': 2}
['time:timestamp']


In [27]:
def train_evaluate(config):
    trained_net = train_hgnn(config, output_cat=outputcat, output_real=outputreal, epochs=80)
    return test_hgnn(trained_net, output_cat=outputcat, output_real=outputreal)

In [28]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "hid", "type": "choice", "values": [16, 64, 128], "value_type": "int", "is_ordered" : True,"sort_values":False},
        #{"name": "layers", "type": "choice", "values": [2, 3, 4, 5], "value_type": "int", "is_ordered" : True, "sort_values":False},
        {"name": "layers", "type": "choice", "values": [2], "value_type": "int", "is_ordered" : True, "sort_values":False},
        {"name": "lr", "type": "range", "bounds": [1e-4, 1e-1], "value_type": "float", "log_scale": True},
        {"name": "batch_size", "type": "choice", "values": [32, 64, 128, 256], "value_type": "int", "is_ordered" : True,"sort_values":False}, 
        {"name": "heads", "type": "choice", "values": [1,2], "value_type": "int", "is_ordered" : True,"sort_values":False},
        #{"name": "heads", "type": "choice", "values": [1], "value_type": "int", "is_ordered" : True,"sort_values":False},
        
        #{"name": "aggregation", "type" : "choice", "values" :["sum", "mean", "max"], "value_type" : "str"}
        {"name": "aggregation", "type" : "choice", "values" :["max"], "value_type" : "str"},
     
    ],
  
    evaluation_function=train_evaluate,
    objective_name='AVG_total_loss',
    arms_per_trial=1,
    minimize = True,
    random_seed = 123,
    total_trials = 20
)

print(best_parameters)
means, covariances = values
print(means)
print(experiment)

[INFO 02-13 11:25:00] ax.service.utils.instantiation: Choice parameter layers contains only one value, converting to a fixed parameter instead.
[INFO 02-13 11:25:00] ax.service.utils.instantiation: Choice parameter aggregation contains only one value, converting to a fixed parameter instead.
[INFO 02-13 11:25:00] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[ChoiceParameter(name='hid', parameter_type=INT, values=[16, 64, 128], is_ordered=True, sort_values=False), FixedParameter(name='layers', parameter_type=INT, value=2), RangeParameter(name='lr', parameter_type=FLOAT, range=[0.0001, 0.1], log_scale=True), ChoiceParameter(name='batch_size', parameter_type=INT, values=[32, 64, 128, 256], is_ordered=True, sort_values=False), ChoiceParameter(name='heads', parameter_type=INT, values=[1, 2], is_ordered=True, sort_values=False), FixedParameter(name='aggregation', parameter_type=STRING, value='max')], parameter_constraints=[]).
[INFO 02-13 11:25:00] ax.modelbri

{'hid': 16, 'lr': 0.005537727881833665, 'batch_size': 64, 'heads': 2, 'layers': 2, 'aggregation': 'max'}
Epoch: 0



KeyboardInterrupt: 

In [ ]:
config = {'lr': 0.013969377507803457, 'batch_size': 64, 'heads': 1, 'hid': 128, 'layers': 2, 'aggregation': 'max'}

In [ ]:
train_evaluate(config)

In [ ]:
from ax.service.utils.report_utils import exp_to_df

results = exp_to_df(experiment)

In [ ]:
results.sort_values(by="AVG_total_loss")

In [ ]:
results = results.sort_values(by="AVG_total_loss")

In [ ]:
results.to_csv("results/BPI13O.csv", sep=",")